# PACKAGE INSTALATION




In [ ]:
!pip install pymupdf
!pip install sentence-transformers
!pip install langchain
!pip install transformers
!pip install chromadb
!pip install langchain_community
!pip install tiktoken
!pip install python-dotenv
!pip install langchain-groq
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 29.4 MB/s eta 0

# Mounting Drive

In [35]:
from google.colab import drive
drive.mount('/content/drive/')

%cd '/content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/'

!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main
chroma_db	DocumentChunking.py  PDF_FILES	  ResponseGenerator.py
COLAB_NOTEBOOK	main.py		     __pycache__  VectorEmbedding.py


In [36]:
#@title .gitignore
%%writefile .gitignore
.env
*__pycache__

Overwriting .gitignore


In [ ]:
#@title DocumentChunking.py
%%writefile DocumentChunking.py

import os
import hashlib
import traceback
import fitz
from typing import List,Dict,Tuple
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

class DocumentChunking:
  def __init__(self, PDF_Path, chunk_size, chunk_overlap):
    self.PDF_Path = PDF_Path
    self.Chunk_Size = chunk_size
    self.Chunk_Overlap = chunk_overlap

  def PDF_Hashing(self):
    try:
      hasher = hashlib.md5()
      with open(self.PDF_Path, 'rb') as fl:
        while chunk:=fl.read(8192):
          hasher.update(chunk)
        return hasher.hexdigest()
    except Exception as ex:
      print(f"Exception: {ex}")
      print(traceback.print_exc())
      raise ValueError(f"Exception to hash PDF {self.PDF_Path}: {ex}")


  def Chunking_Document(self):
    try:

      if not os.path.exists(self.PDF_Path):
        raise FileNotFoundError(f"Given file {self.PDF_Path} not found")
      hash_doc = self.PDF_Hashing()

      Recursive_text_splitter = RecursiveCharacterTextSplitter(
          chunk_size = self.Chunk_Size,
          chunk_overlap = self.Chunk_Overlap,
          length_function = len)


      doc_chunks = []
      with fitz.open(self.PDF_Path) as document:
        for idx, page in enumerate(document):
          texts_in_page = page.get_text()
          if texts_in_page.strip():
            page_chunks = Recursive_text_splitter.split_text(texts_in_page)
            for idx_chnk, chunk_content in enumerate(page_chunks):
              doc_chunks.append({
                "content":chunk_content,
                "metadata":{
                  "document_id":hash_doc,
                  "Source_File":os.path.basename(self.PDF_Path),
                  "Page_Number":idx+1,
                  "Chunk_ID": f"{hash_doc}-{idx}-{idx_chnk}" } })
        if not doc_chunks:
          raise ValueError(f"No text extracted from {self.PDF_Path}")
        else:
          return hash_doc, doc_chunks

    except Exception as ex:
      print(f"Exception occured in chunking: {ex}")
      print(traceback.print_exc())
      return None, []






Writing DocumentChunking.py


In [ ]:
#@title VectorEmbedding.py
%%writefile VectorEmbedding.py

import chromadb
import hashlib
from typing import List, Dict, Tuple
from sentence_transformers import SentenceTransformer
from chromadb.api.types import EmbeddingFunction

class VectorEmbedding:
  def __init__(self, model_name,chromadb_Path, collection_name):
    self.model = SentenceTransformer(model_name)
    self.client = chromadb.PersistentClient(path=chromadb_Path)
    self.Collection_name = collection_name
    self.collection = self._get_or_create_collection()

    print(f"Chroma DB Initialized at {chromadb_Path}, collection {collection_name}")

  def _get_or_create_collection(self):
    class ChormaEmbeddingFunction(EmbeddingFunction):
      def __init__(self, model):
        self.model = model

      def __call__(self,texts: List[str]):
        return self.model.encode(texts).tolist()

    return self.client.get_or_create_collection(
        name = self.Collection_name,
        embedding_function = ChormaEmbeddingFunction(self.model) )

  def Embedding_Generator(self,text):
    return self.model.encode(text).tolist()

  def Add_chunk_To_VectorDB(self, documents):
    try:
      if not documents:
        return []
      ids=[]
      contents=[]
      metadatas=[]
      for doc in documents:
        ids.append(doc['metadata']['Chunk_ID'])
        contents.append(doc['content'])
        metadatas.append(doc['metadata'])

      self.collection.add(documents=contents, metadatas=metadatas, ids=ids)
      return ids
    except Exception as ex:
      print(f"Execption in Add_chunk_To_VectorDB: {ex}")
      return []

  def get_document_hash(self,hash_document):
    hash_result = self.collection.get(where={"document_id":hash_document},limit=1)
    return hash_result['ids'] if hash_result and hash_result['ids'] else[]

  def search(self, input_query,k_val=3):
    try:
      input_query_embedding = self.model.encode(input_query).tolist()
      search_result = self.collection.query(
          query_embeddings = [input_query_embedding],
          n_results = k_val,
          include=['documents','metadatas','distances'] )

      document_retrived = []
      if search_result and search_result['documents'] and search_result['metadatas']:
        for indx in range(len(search_result['documents'][0])):
          document_content = search_result['documents'][0][indx]
          document_metadata = search_result['metadatas'][0][indx]
          document_distance = search_result['distances'][0][indx]

          document_retrived.append({
              "content":document_content,
              "metadata":document_metadata,
              "distance":document_distance })

        print(f"Retrived {len(document_retrived)} documents from chroma db")

      return document_retrived

    except Exception as ex:
      print(f"Exception in search: {ex}")
      print(traceback.print_exc())
      return []




Writing VectorEmbedding.py


In [ ]:
#@title ResponseGenerator.py
%%writefile ResponseGenerator.py

import os
from typing import List, Dict
from groq import Groq

class ResponseGenerator:
  def __init__(self, api_key, model_name):
    self.api_key = api_key
    self.Model_Name = model_name
    self.System_Message ="You are Retrival Auggumented Retreival Chat BOT agent for Fly Kite Airlies for HR Policy document"


  def Response_Genrator(self, prompt):
    try:
      client = Groq(api_key=self.api_key)
      completion = client.chat.completions.create(
          model=self.Model_Name,
          messages=[{"role":"system","content": self.System_Message},
                    {"role":"user","content":prompt}],
          temperature=0.1,max_completion_tokens=None,
          top_p=1,stream=True,stop=None)
      response = ""
      for chunk in completion:
        if chunk.choices:
          content = chunk.choices[0].delta.content or ""
          response += content
      return response

    except Exception as ex:
      print(f"Exception in Response Generator: {ex}")
      print(traceback.print_exc())
      return f"Unable to retrive data for query \n {ex}"

Writing ResponseGenerator.py


In [ ]:
#@title main.py
%%writefile main.py

import os
import sys
import argparse
import logging
import warnings
import json
from typing import List,Dict
from dotenv import load_dotenv

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_VISIBLE_DEVICES"]=""
warnings.filterwarnings("ignore")
logging.getLogger("tensorflow").setLevel(logging.ERROR)

try:
  base_path = os.path.abspath((os.path.dirname(__file__)))
except:
  base_path = os.path.join(os.getcwd())

print(f"Base_path {base_path}")
load_dotenv(os.path.join(os.getcwd(),'.env'))
groq_api_key = os.getenv('GROQ_API_KEY')
chunk_size=int(os.getenv('CHUNK_SIZE'))
chunk_overlap=int(os.getenv('CHUNK_OVERLAP'))
sentence_transformer_model = os.getenv('SENTENCE_TRANSFORMER_MODEL')
groq_ll_model=os.getenv('GROQ_LLM_MODEL')
pdf_path=os.path.join(base_path,os.getenv('PDF_FILE_NAME'))
chroma_db_path= os.path.join(base_path,os.getenv('CHROMA_DB_PATH'))
collection_name="FlyKite_Airlines"
hf_token=os.getenv('HF_TOKEN')

prompt_instruction = "\n Answer: Provide a clear responde to the Question. incorporating relevant details from the provided context. Include the metadata like document_id, source_file, page_number for each document used in your response"

print(f"PDF PATH: {pdf_path}")
print(f"chroma_db_path: {chroma_db_path}")


parser = argparse.ArgumentParser(description="To Run a Specific Job in pipeline")

parser.add_argument('--job', type=str, required=True,
                    choices=['rag-build','search','hosting'],
                    help='jobs to execute')
parser.add_argument('--raw',action='store_true',
                    help='Display raw retrieved documents in JSON format without feeding to LLM')

args = parser.parse_args()
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
from DocumentChunking import DocumentChunking
from VectorEmbedding import VectorEmbedding
from ResponseGenerator import ResponseGenerator

vectorDBObj = VectorEmbedding(sentence_transformer_model,chroma_db_path,collection_name)

if args.job == 'rag-build':
  if not os.path.exists(pdf_path):
    raise FileNotFoundError(f"File {pdf_path} not found")


  chunker = DocumentChunking(pdf_path,chunk_size,chunk_overlap)
  doc_hash_id = chunker.PDF_Hashing()
  if doc_hash_id and vectorDBObj.get_document_hash(doc_hash_id):
    print(f"Document {pdf_path} already chunked and embbeded in chroma vector db")
  else:
    doc_id, chunks = chunker.Chunking_Document()
    print(f"doc_id {doc_id}")
    print(f"Chunk: {chunks}")
    print(f"{len(chunks)}")
    if not chunks:
      print(f"No Chunks generated for the {pdf_path}")
    else:
      chunk_ids = vectorDBObj.Add_chunk_To_VectorDB(chunks)
      print(f"Added {len(chunk_ids)} chunks in chromaDB")


elif args.job == 'search':
  user_query = input("Enter your Query: ")
  retrived_documents = vectorDBObj.search(user_query)
  #print(retrived_documents)
  if not retrived_documents:
    print("No Document retrived")
  else:
    retrieved_json = [{ "document_number": indx+1,
            "content": doc['content'],
            "metadata":doc['metadata'],
            "distance":doc['distance'] }for indx, doc in enumerate(retrived_documents) ]
    if args.raw:
      print(f"\n Retrived Documents:\n")
      print(json.dumps(retrieved_json, indent=2))
    else:
      print(f"\n Retrived Documents:\n")
      print(json.dumps(retrieved_json, indent=2))
      context = "\n".join([doc['content'] for doc in retrived_documents])
      prompt = f"context: {context} \n\n Question: {user_query}"
      llm_obj = ResponseGenerator(groq_api_key, groq_ll_model)
      response = llm_obj.Response_Genrator(prompt)
      print(f"Generated Response..\n {response}")

elif args.job == 'hosting':
  from HostingInHuggingFace import HostingInHuggingFace
  Hosting_Obj = HostingInHuggingFace(base_path,hf_token)
  Hosting_Obj.ToRunPipeline()



Writing main.py


In [ ]:
#@title Chunking & Vector Embedding main function execution
!export PYTHONWARNINGS="ignore"
!python main.py --job rag-build

Base_path /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main
PDF PATH: /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/PDF_FILES/Flykite Airlines_ HRP.pdf
chroma_db_path: /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/chroma_db
E0000 00:00:1759205133.228514    4935 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759205133.235691    4935 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759205133.253964    4935 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759205133.254023    4935 computation_placer.cc:177] computation placer already registered. Please 

## Question 01 leave and attendance policy

In [ ]:
!python main.py --job search

Base_path /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main
PDF PATH: /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/PDF_FILES/Flykite Airlines_ HRP.pdf
chroma_db_path: /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/chroma_db
E0000 00:00:1759205196.412670    5210 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759205196.419691    5210 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759205196.437925    5210 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759205196.437999    5210 computation_placer.cc:177] computation placer already registered. Please 

## Question #02 Leave Accrual, Carry-Over, and Forfeiture ?

In [ ]:
!python main.py --job search

Base_path /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main
PDF PATH: /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/PDF_FILES/Flykite Airlines_ HRP.pdf
chroma_db_path: /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/chroma_db
E0000 00:00:1759205263.303892    5505 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759205263.311093    5505 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759205263.329016    5505 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759205263.329069    5505 computation_placer.cc:177] computation placer already registered. Please 

## Question #03 Allowable Expenses and Reimbursement Procedures

In [ ]:
!python main.py --job search

Base_path /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main
PDF PATH: /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/PDF_FILES/Flykite Airlines_ HRP.pdf
chroma_db_path: /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/chroma_db
E0000 00:00:1759205328.755084    5791 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759205328.767629    5791 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759205328.797376    5791 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759205328.797436    5791 computation_placer.cc:177] computation placer already registered. Please 

## Question #04 Customer Data Protection and Privacy6. Customer Data Protection and Privacy ?

In [ ]:
!python main.py --job search

Base_path /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main
PDF PATH: /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/PDF_FILES/Flykite Airlines_ HRP.pdf
chroma_db_path: /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/chroma_db
E0000 00:00:1759205389.745817    6059 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759205389.753047    6059 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759205389.771478    6059 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759205389.771528    6059 computation_placer.cc:177] computation placer already registered. Please 

## Question #05 Attendance and Absence Management ?

In [ ]:
!python main.py --job search

Base_path /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main
PDF PATH: /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/PDF_FILES/Flykite Airlines_ HRP.pdf
chroma_db_path: /content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/12_CAPSTONE_PROJECT/RAG_FlyKite_Airlnes/main/chroma_db
E0000 00:00:1759205446.712459    6317 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759205446.720072    6317 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759205446.737704    6317 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759205446.737755    6317 computation_placer.cc:177] computation placer already registered. Please 